In [ ]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 81.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 req

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [ ]:
huggingface_dataset_name = "sharad/chatgpt-paraphrases-simple"

dataset = load_dataset(huggingface_dataset_name)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('ENGLISH INPUT:')
    print(dataset['train']['s1'][index])
    print(dash_line)
    print('BASELINE paraphrase:')
    print(dataset['train']['s2'][index])
    print()



---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
ENGLISH INPUT:
Is there a method to boost my internet connection speed when using a VPN?
---------------------------------------------------------------------------------------------------
BASELINE VIETNAMESE:
Are there any techniques to increase my internet speed while using a VPN?

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
ENGLISH INPUT:
Can you describe your initial sexual encounter?
---------------------------------------------------------------------------------------------------
BASELINE VIETNAMESE:
How would you characterize your initial sexual experience?



In [ ]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
sentence = "What is the time, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([ 363,   19,    8,   97,    6, 3059,   58,    1])

DECODED SENTENCE:
What is the time, Tom?


In [ ]:
for i, index in enumerate(example_indices):
    s1 = dataset['train']['s1'][index]
    s2 = dataset['train']['s2'][index]

    inputs = tokenizer(s1, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{s1}')
    print(dash_line)
    print(f'BASELINE paraphrase:\n{s2}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Is there a method to boost my internet connection speed when using a VPN?
---------------------------------------------------------------------------------------------------
BASELINE Vietnamese:
Are there any techniques to increase my internet speed while using a VPN?
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
no

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Can you describe your initial sexual encounter?
----------------------------------------------------------------------------------

In [ ]:
for i, index in enumerate(example_indices):
    s1 = dataset['train']['s1'][index]
    s2 = dataset['train']['s2'][index]

    prompt = f"""
Paraphrase the statement

{s1}

After paraphrasing:
    """

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=100,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE paraphrase:\n{s2}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Paraphrase the statement

Is there a method to boost my internet connection speed when using a VPN?

After paraphrasing:
    
---------------------------------------------------------------------------------------------------
BASELINE Vietnamese:
Are there any techniques to increase my internet speed while using a VPN?
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
Is there a method to increase my internet connection speed when using a VPN?

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Pa

In [ ]:
def make_prompt(example_indices_full, example_index_to_para):
    prompt = ''
    for index in example_indices_full:
        s1 = dataset['train']['s1'][index]
        s2 = dataset['train']['s2'][index]

        # The stop sequence '{paraphrase}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Paraphrase the statement below

{s1}

Here is its paraphase:
{s2}


"""

    s1 = dataset['train']['s1'][example_index_to_para]

    prompt += f"""
Paraphrase the statement below

{s1}

Here is its paraphase:
"""

    return prompt

In [ ]:
example_indices_full = [40]
example_index_to_para = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_para)

print(one_shot_prompt)


Paraphrase the statement below

Is there a method to boost my internet connection speed when using a VPN?

Here is its paraphase:
Are there any techniques to increase my internet speed while using a VPN?



Paraphrase the statement below

Can you describe your initial sexual encounter?

Here is its paraphase:



In [ ]:

s2 = dataset['train']['s2'][example_index_to_para]


inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE paraphrase:\n{s2}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE VIETNAMESE:
How would you characterize your initial sexual experience?

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
I was a virgin.


In [ ]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Paraphrase the statement below

Is there a method to boost my internet connection speed when using a VPN?

Here is its paraphase:
Are there any techniques to increase my internet speed while using a VPN?



Paraphrase the statement below

What can be inferred about me based on my Capricorn Sun, Capricorn Moon, and Capricorn Rising signs in astrology?

Here is its paraphase:
How does my Capricorn Sun, Capricorn Moon, and Capricorn Rising signs in astrology define my personality?



Paraphrase the statement below

When do you use シ instead of し?

Here is its paraphase:
In what situations is シ used instead of し?



Paraphrase the statement below

Can you describe your initial sexual encounter?

Here is its paraphase:



In [ ]:
s2 = dataset['train']['s2'][example_index_to_summarize]


inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE paraphrase:\n{s2}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE VIETNAMESE:
How would you characterize your initial sexual experience?

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
I was a virgin.


# **Fine tune Generative models**

In [ ]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [ ]:
huggingface_dataset_name = "sharad/chatgpt-paraphrases-simple"

dataset = load_dataset(huggingface_dataset_name)

dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['s1', 's2'],
        num_rows: 6286314
    })
})

In [ ]:
#dataset['train'].train_test_split()

In [ ]:
dataset['train'] = dataset['train'].shuffle()

In [ ]:
train_set = dataset['train'][:1000]
val_set = dataset['train'][1000:1200]
test_set = dataset['train'][1200:1500]

In [ ]:
len(train_set['s1'])

1000

In [ ]:
from datasets import Dataset
train_set = Dataset.from_dict(train_set)
val_set = Dataset.from_dict(val_set)
test_set = Dataset.from_dict(test_set)

In [ ]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [ ]:
index = 200

s1 = test_set['s1'][index]
s2 = test_set['s2'][index]

prompt = f"""
Paraphrase the following sentence.

{s1}

New sentence:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN PARAPHRASE:\n{s2}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Paraphrase the following sentence.

What is Enlightenment? How can I be Enlightened?

New sentence:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN PARAPHRASE:
What is the definition of Enlightenment? How can I reach it?

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
What is Enlightenment? How can I be Enlightened?


In [ ]:
def tokenize_function(example):
    start_prompt = f'''Paraphrase this sentence.

What does it mean to be loved without condition?

New Sentence:
How would you describe the experience of being loved unconditionally?



Paraphrase this sentence.\n\n'''
    end_prompt = '\n\nNew sentence: '
    prompt = [start_prompt + s1 + end_prompt for s1 in example["s1"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["s2"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example



In [ ]:
# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_train = train_set.map(tokenize_function, batched=True)
tokenized_train = tokenized_train.remove_columns(['s1', 's2'])
tokenized_val = val_set.map(tokenize_function, batched=True)
tokenized_val = tokenized_val.remove_columns(['s1', 's2'])
tokenized_test = test_set.map(tokenize_function, batched=True)
tokenized_test = tokenized_test.remove_columns(['s1', 's2'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
''' Đoạn này là giảm size data nè giảm 100 lần
tokenized_train = tokenized_train.filter(lambda example, index: index % 100 == 0, with_indices=True)
tokenized_val = tokenized_val.filter(lambda example, index: index % 100 == 0, with_indices=True)
tokenized_test = tokenized_test.filter(lambda example, index: index % 100 == 0, with_indices=True)
"""

SyntaxError: incomplete input (<ipython-input-17-3379ff8fc47c>, line 1)

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_train.shape}")
print(f"Validation: {tokenized_val.shape}")
print(f"Test: {tokenized_test.shape}")



Shapes of the datasets:
Training: (1000, 2)
Validation: (200, 2)
Test: (300, 2)


In [ ]:
output_dir = f'./paraphrase-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,42.000000


TrainOutput(global_step=1, training_loss=42.0, metrics={'train_runtime': 3.4294, 'train_samples_per_second': 2.333, 'train_steps_per_second': 0.292, 'total_flos': 5478058819584.0, 'train_loss': 42.0, 'epoch': 0.01})

In [ ]:
index = 200
s1 = test_set[index]['s1']
human_baseline_paraphrase = test_set[index]['s2']

prompt = f"""
Paraphrase this sentence.

{s1}

New Sentence:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids.to('cuda:0')

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = pretrained_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN paraphrase:\n{human_baseline_paraphrase}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
What is the definition of Enlightenment? How can I reach it?
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
Enlightenment is the process of gaining knowledge and understanding.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
Enlightenment


In [ ]:
instruct_model = trainer.model
instruct_model = instruct_model.to('cpu')

In [ ]:

device = torch.device("cpu")

In [ ]:
test_set[200]

{'s1': 'What is Enlightenment? How can I be Enlightened?',
 's2': 'What is the definition of Enlightenment? How can I reach it?'}

In [ ]:
bleu = evaluate.load('bleu')

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
  sentence1s = test_set['s1'][0:10]
  human_baseline = test_set['s2'][0:10]

  original_model_paraphrases = []
  instruct_model_paraphrases = []

  for _, s1 in enumerate(sentence1s):
    prompt = f"""
Paraphrase this sentence.

{s1}

New Sentence: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_paraphrases.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_paraphrases.append(instruct_model_text_output)

  zipped_paraphrases = list(zip(human_baseline, original_model_paraphrases, instruct_model_paraphrases))

  df = pd.DataFrame(zipped_paraphrases, columns = ['human_baseline_paraphrases', 'original_model_paraphrases', 'instruct_model_paraphrases'])
  df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,How would you describe the experience of being...,What does it mean to be loved without condition?,What does it mean to be loved without condition?
1,What is a feasible monthly budget and income f...,A recently certified diving instructor residin...,A recently certified diving instructor residin...
2,"How do vision, mission, and values factor into...","What is the role of vision, mission and values...","What is the role of vision, mission and values..."
3,"A diaphragm or piston can act as a ""brake"" if ...",If the pressure above the cab suddenly increas...,If the pressure above the cab suddenly increas...
4,By what means can a solitary word be transform...,"Trump's ""three words: jobs, jobs, jobs"" is a p...","Trump's ""three words: jobs, jobs, jobs"" is a p..."
5,What leads to the significant amount of anti-I...,Israel is a country that is not a country that...,Israel is a country that is not a country that...
6,The slip frequency decreases and the current i...,The slip frequency decreases and the current i...,The slip frequency decreases and the current i...
7,What is the procedure to measure kinetic energy?,The amount of kinetic energy is determined by ...,The amount of kinetic energy is determined by ...
8,What is the suggested frequency for performing...,How often should range-of-motion exercises be ...,How often should range-of-motion exercises be ...
9,"A young superfan named Gianni Zandri, believed...","Gianni Zandri, a self-proclaimed superfan who ...","Gianni Zandri, a self-proclaimed superfan who ..."


In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_paraphrases,
    references=human_baseline[0:len(original_model_paraphrases)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_paraphrases,
    references=human_baseline[0:len(instruct_model_paraphrases)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.5025772834940796, 'rouge2': 0.31463082668965026, 'rougeL': 0.3975023327609508, 'rougeLsum': 0.40052464849543573}
INSTRUCT MODEL:
{'rouge1': 0.4978295336603667, 'rouge2': 0.31306332325774, 'rougeL': 0.395767796543656, 'rougeLsum': 0.3990994490434516}


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [ ]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 884736
all model parameters: 248462592
percentage of trainable model parameters: 0.36%


In [ ]:
output_dir = f'./peft-dialogue-paraphrase-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=50,
    logging_steps=1,
    max_steps=50
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_train,
)

In [ ]:
peft_trainer.train()

peft_model_path="./peft-dialogue-paraphrase-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss


Step,Training Loss
1,41.000000
2,33.750000
3,30.250000
4,29.000000
5,25.625000
6,24.125000
7,22.125000
8,19.250000
9,17.500000
10,15.375000


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [ ]:
instruct_model = peft_trainer.model.to('cpu')

In [ ]:
  sentence1s = test_set['s1'][0:10]
  human_baseline = test_set['s2'][0:10]

  original_model_paraphrases = []
  instruct_model_paraphrases = []

  for _, s1 in enumerate(sentence1s):
    prompt = f"""
Paraphrase this sentence.

What does it mean to be loved without condition?

New Sentence:
How would you describe the experience of being loved unconditionally?



Paraphrase this sentence.

{s1}

New Sentence: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_paraphrases.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_paraphrases.append(instruct_model_text_output)

  zipped_paraphrases = list(zip(human_baseline, original_model_paraphrases, instruct_model_paraphrases))

  df = pd.DataFrame(zipped_paraphrases, columns = ['human_baseline_paraphrases', 'original_model_paraphrases', 'instruct_model_paraphrases'])
  df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,How would you describe the experience of being...,the experience of being loved unconditionally?,The experience of being loved unconditionally.
1,What is a feasible monthly budget and income f...,The monthly income for the dive instructor wou...,If a new a dive instructor residing in Koh Tao...
2,"How do vision, mission, and values factor into...",The strategic management process is a combinat...,The strategic management process is a process ...
3,"A diaphragm or piston can act as a ""brake"" if ...","The pressure above the cab is a ""brake"" that c...","The pressure above the cab can be a ""brake."""
4,By what means can a solitary word be transform...,"By using a word that is a new word, you can sa...","When you repeat a word, you can't repeat a sin..."
5,What leads to the significant amount of anti-I...,Having a restraining of sorts is a sign of a r...,The negative views of Israel are a direct resu...
6,The slip frequency decreases and the current i...,The acceleration of the motor and the current ...,The slip frequency increases and the current i...
7,What is the procedure to measure kinetic energy?,The amount of kinetic energy can be determined...,measuring the amount of kinetic energy
8,What is the suggested frequency for performing...,Your body will be able to move around with you...,The range of motion should be performed on non...
9,"A young superfan named Gianni Zandri, believed...",'Birdman' Andersen',Gianni Zandri is a self-professed superfan who...


In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_paraphrases,
    references=human_baseline[0:len(original_model_paraphrases)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_paraphrases,
    references=human_baseline[0:len(instruct_model_paraphrases)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.39968176159352636, 'rouge2': 0.24545454545454545, 'rougeL': 0.3154077186430127, 'rougeLsum': 0.3112939620292562}
INSTRUCT MODEL:
{'rouge1': 0.4769659296652652, 'rouge2': 0.26639125707366496, 'rougeL': 0.40963688444929136, 'rougeLsum': 0.40902120975726197}


In [ ]:
test_set['s1'][1]

'What would be a practical monthly income and budget for a recently certified diving instructor residing in Koh Tao?'

In [ ]:
test_set['s2'][0]

'How would you describe the experience of being loved unconditionally?'

In [ ]:
original_model_paraphrases[1]

'The monthly income for the dive instructor would be: $.0 per month.'

In [ ]:
instruct_model_paraphrases[1]

'If a new a dive instructor residing in Koh Tao, a new a resident residing in Koh Tao, a new a resident would have a minimum of $80,000 in salary.'

In [ ]:
original_model_results = bleu.compute(
    predictions=original_model_paraphrases,
    references=human_baseline[0:len(original_model_paraphrases)],
)

instruct_model_results = bleu.compute(
    predictions=instruct_model_paraphrases,
    references=human_baseline[0:len(instruct_model_paraphrases)],
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'bleu': 0.14367416512328487, 'precisions': [0.41843971631205673, 0.22900763358778625, 0.1487603305785124, 0.10714285714285714], 'brevity_penalty': 0.7267672993071735, 'length_ratio': 0.7580645161290323, 'translation_length': 141, 'reference_length': 186}
INSTRUCT MODEL:
{'bleu': 0.15620374159426711, 'precisions': [0.3973509933774834, 0.2198581560283688, 0.16030534351145037, 0.10743801652892562], 'brevity_penalty': 0.793114182676225, 'length_ratio': 0.8118279569892473, 'translation_length': 151, 'reference_length': 186}


# Align with human feedback